In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics


#run cleaning.ipynb to get clean data sets from the test and train csv provided by Kaggle.
test_data = pd.read_csv("test_data_clean.csv")
train_data = pd.read_csv("train_data_clean.csv")

In [ ]:
# train and test split

#Remove prices for blind testing. Save for fitting. Save copy as saleprices_actual for RMSE evaluation later
y_train = train_data.SalePrice.values
saleprices_actual = train_data.SalePrice.values
train_data.drop(['SalePrice'], axis = 1, inplace = True)

x_train = train_data.values
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.40,random_state=0)

In [ ]:
#Cross validation analysis
train_dataset = lgb.Dataset(x_train, y_train)
test_dataset = lgb.Dataset(x_test, y_test)
params = {
    "objective" : "regression",
    "max_depth" : 7,
    "task" : "train",
    "learning_rate" : 0.1,
    "subsample" : 0.8,
    "boosting_type" : "gbdt",
    "num_leaves" : 31
}
booster = lgb.train(params,
                    train_set=train_dataset, valid_sets=(test_dataset,),
                    num_boost_round=10)

from sklearn.metrics import r2_score
test_predictions = booster.predict(x_test)
train_predictions = booster.predict(x_train)

print("\nTest  R2 Score : %.2f"%r2_score(y_test, test_predictions))
print("Train R2 Score : %.2f"%r2_score(y_train, train_predictions))

In [ ]:
from sklearn.metrics import r2_score
test_predictions = booster.predict(x_test)
train_predictions = booster.predict(x_train)

print("\nTest  R2 Score : %.2f"%r2_score(y_test, test_predictions))
print("Train R2 Score : %.2f"%r2_score(y_train, train_predictions))

In [ ]:
#LightGBM modeling
model = lgb.LGBMClassifier(learning_rate=0.1,max_depth=7,task='train',
                           objective='regression',subsample=0.8, boosting_type='gbdt',
                           num_leaves = 31)
model.fit(x_train,y_train,eval_set=[(x_train, y_train)])

In [ ]:
#Visualization of price predictions
ID = train_data.Id
price_predictions = model.predict(train_data)
Predictions = pd.DataFrame({'ID' : ID, 'Price Prediction' : price_predictions})
pd.set_option("display.max_rows", None, "display.max_columns", None)
Predictions

In [ ]:
#Calculate RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
y_train.shape
price_predictions.shape
sqrt(mean_squared_error(saleprices_actual, price_predictions)) 